## --- Day 20: Trench Map ---
Start with the original input image and apply the image enhancement algorithm twice, being careful to account for the infinite size of the images. How many pixels are lit in the resulting image?

In [19]:
class ImageEnhancer:
    def __init__(self, algorithm, input_img):
        self.algorithm = algorithm
        self.input_img = input_img
        self.image = self.ImageMap(input_img)

    @classmethod
    def from_file(cls, filename):
        with open(filename) as f:
            algorithm = f.readline().strip()
            _ = f.readline() # skip blank line
            input_img = []
            for line in f.readlines():
                input_img.append(list(line.strip()))

        return ImageEnhancer(algorithm, input_img)

    def enhance(self, iterations=1, verbose=False):
        def _enhance(verbose=False):
            enhanced_input = []
            for r in range(-1, self.image.last_row+2):
                row = []
                for c in range(-1, self.image.last_col+2):
                    pixel_index = self.image.get_pixel_index(r, c)
                    row.append(self.algorithm[pixel_index])
                if verbose:
                    print(" ".join(row))
                enhanced_input.append(row)

            # The background "fill" pixel that extends from the canvas
            # to infinity in all directions may flip depending on the
            # algorithm (specifically, the 0 and last indices)
            background = self.algorithm[0] if self.image.bg_pixel == "." else self.algorithm[-1]
            self.image = self.ImageMap(enhanced_input, background)

        for _ in range(iterations):
            _enhance(verbose=verbose)

    
    class ImageMap:
        def __init__(self, input_img, background="."):
            self.pixel_map = {}
            for r in range(len(input_img)):
                for c in range(len(input_img[r])):
                    self.pixel_map[(r, c)] = input_img[r][c]
            self.bg_pixel = background
            self.last_row = r
            self.last_col = c
        
        def get_pixel_index(self, row, col):
            # Calculate the index as a binary number derived from all 9 adjacent pixels (including itself)
            adjacent_pixels = []
            for r in range(row-1, row+2):
                for c in range(col-1, col+2):
                    adjacent_pixels.append((r, c))
            
            result = 0
            for r, c in adjacent_pixels:
                pixel_value = self.pixel_map.get((r, c), self.bg_pixel)
                result = (result << 1) + (1 if pixel_value == "#" else 0)

            return result

        def count_active_pixels(self):
            return len([v for v in self.pixel_map.values() if v == "#"])


In [20]:
# Example
ex1 = ImageEnhancer.from_file("./inputs/Day20ex.txt")
ex1.image.count_active_pixels()
ex1.enhance(verbose=True)
ex1.enhance(verbose=True)
assert 35 == ex1.image.count_active_pixels()

. # # . # # .
# . . # . # .
# # . # . . #
# # # # . . #
. # . . # # .
. . # # . . #
. . . # . # .
. . . . . . . # .
. # . . # . # . .
# . # . . . # # #
# . . . # # . # .
# . . . . . # . #
. # . # # # # # .
. . # . # # # # #
. . . # # . # # .
. . . . # # # . .


In [21]:
# Part 1 solution
p1 = ImageEnhancer.from_file("./inputs/Day20.txt")
p1.enhance(iterations=2)
p1.image.count_active_pixels()

5081

## --- Part Two ---

Start again with the original input image and apply the image enhancement algorithm 50 times. How many pixels are lit in the resulting image?

In [26]:
# Example
ex2 = ImageEnhancer.from_file("./inputs/Day20ex.txt")
ex2.enhance(iterations=50)
assert 3351 == ex2.image.count_active_pixels()


In [27]:
# Part 2 solution
p2 = ImageEnhancer.from_file("./inputs/Day20.txt")
p2.enhance(iterations=50)
p2.image.count_active_pixels()

15088